<a href="https://colab.research.google.com/github/z4n0/LessonsExtractor/blob/main/LessonsExtractorRecovered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%ls /content/drive/MyDrive/lessonsExtractor

/content/drive/MyDrive/lessonsExtractor@


NB: TURN ON THE T4 GPU

In [ ]:
!pip install opencv-python

In [ ]:
%cd drive/MyDrive/lessonsExtractor/

/content/drive/.shortcut-targets-by-id/1bQb8gi6zCwEeUgzwHQLPETmzfh9dMdVq/lessonsExtractor


In [ ]:
%ls

 20240404_converted.docx       lez15_converted.docx
 gpu-20240430_converted.docx   lez15.mp4
 gpu-20240430.mp4              lez3_converted.docx
 gpu-20240515_converted.docx   lez3_slides_only.docx
 gpu-20240515.mp4              lez6.mp4
 gpu-20240517.mp4              lez9.docx
 gpu-20240529_converted.docx  'Marcello Restelli_s Personal Room-20240312 1333-1_converted.docx'
 gpu-20240529.mp4             'Marcello Restelli_s Personal Room-20240326 1330-1_converted.docx'
 gpu-20240531.mp4              ML20240416_converted.docx
 LessonsExtractor.ipynb        ML20240430_converted.docx
 lez13_converted.docx          ML20240502_converted.docx
 lez13.mp4


#slide extractor

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
%ls #to copy paste the name of the video you want to transcribe

 20240404_converted.docx       lez15_converted.docx
 gpu-20240430_converted.docx   lez15.mp4
 gpu-20240430.mp4              lez3_converted.docx
 gpu-20240515_converted.docx   lez3_slides_only.docx
 gpu-20240515.mp4              lez6.mp4
 gpu-20240517.mp4              lez9.docx
 gpu-20240529_converted.docx  'Marcello Restelli_s Personal Room-20240312 1333-1_converted.docx'
 gpu-20240529.mp4             'Marcello Restelli_s Personal Room-20240326 1330-1_converted.docx'
 gpu-20240531.mp4              ML20240416_converted.docx
 LessonsExtractor.ipynb        ML20240430_converted.docx
 lez13_converted.docx          ML20240502_converted.docx
 lez13.mp4


In [ ]:
%cd /content/drive/MyDrive/lessonsExtractor/

/content/drive/.shortcut-targets-by-id/1bQb8gi6zCwEeUgzwHQLPETmzfh9dMdVq/lessonsExtractor


In [ ]:
input_video_path = 'gpu-20240515.mp4'

# Extract base name and remove extension to form the new output file name
base_name = os.path.basename(input_video_path)
file_name_without_ext = os.path.splitext(base_name)[0]
output_video_path = f"/content/drive/MyDrive/lessonsExtractor/{file_name_without_ext}_converted.mp4"
resolution = "1280x720"
bitrate = "1M"
start_time = "00:00:00"  # Start at 0 minutes

In [ ]:
#duration = "00:30:00"  # Duration of 50 minutes, will stop at the end of the video if it's shorter

# Standard FFmpeg command
#!ffmpeg -ss {start_time} -i {input_video_path} -t {duration} -s {resolution} -b:v {bitrate} -c:v libx264 -c:a aac {output_video_path}

# GPU accelerated version
#!ffmpeg -hwaccel cuda -ss {start_time} -i {input_video_path} -s {resolution} -b:v {bitrate} -c:v h264_nvenc -c:a aac {output_video_path}
#!ffmpeg -i {input_video_path} -ss {start_time} -s {resolution} -b:v {bitrate} -c:v h264_nvenc -c:a aac {output_video_path}

you should see the new converted_video.mp4 here. let's check!

**NB: THE PARAMETER THRESHOLD IN DETECT_SLIDES(....) HAS TO BE FINE TUNED FOR EACH VIDEO**

the higher the value the more selective it becomes

In [ ]:
def sum_time(time1, time2):
    # Convert time to seconds
    def time_to_seconds(time_str):
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s

    # Convert seconds back to time
    def seconds_to_time(seconds):
        h = seconds // 3600
        m = (seconds % 3600) // 60
        s = seconds % 60
        return f"{h:02d}:{m:02d}:{s:02d}"

    total_seconds = time_to_seconds(time1) + time_to_seconds(time2)
    return seconds_to_time(total_seconds)

In [ ]:
def save_frame(frame, timestamp, output_folder='slides'):
    print(sum_time(timestamp, start_time)) ## WHERE sum_time IS DEFINED?
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    filename = f"{output_folder}/slide_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"Saved: {filename}")

def frame_difference(prev_frame, curr_frame):
    # Compute the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, curr_frame)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count

def milliseconds_to_hh_mm_ss(milliseconds):
    hours = int(milliseconds / 3600000)
    minutes = int((milliseconds % 3600000) / 60000)
    seconds = int((milliseconds % 60000) / 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

#trashold is to be fine tuned for each video
def detect_slides(video_path, threshold=420000, skip_frames=30):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = []

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            gray_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            if frame_difference(prev_frame, gray_frame) > threshold:
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp_readable = milliseconds_to_hh_mm_ss(timestamp_ms)
                adjusted_timestamp = sum_time(timestamp_readable, start_time)  # Adjust the timestamp
                slide_changes.append(adjusted_timestamp)
                save_frame(curr_frame, adjusted_timestamp)  # Use adjusted timestamp
                prev_frame = gray_frame
        frame_count += 1

    cap.release()
    return slide_changes

#input_video_path = output_video_path
slide_changes = detect_slides(input_video_path)
print(f"Slide changes detected at: {slide_changes}")

00:00:01
Saved: slides/slide_00:00:01.jpg
00:00:02
Saved: slides/slide_00:00:02.jpg
00:00:03
Saved: slides/slide_00:00:03.jpg
00:00:04
Saved: slides/slide_00:00:04.jpg
00:00:06
Saved: slides/slide_00:00:06.jpg
00:00:07
Saved: slides/slide_00:00:07.jpg
00:00:08
Saved: slides/slide_00:00:08.jpg
00:00:09
Saved: slides/slide_00:00:09.jpg
00:00:10
Saved: slides/slide_00:00:10.jpg
00:00:12
Saved: slides/slide_00:00:12.jpg
00:00:13


KeyboardInterrupt: 

##NEW OPTIMIZED FUNCTION BY LessonExtractorAI (IT SEEMS TO WORK VERY WELL)
Instead of just using frame difference, you could incorporate detection based on more stable features, such as outlines or text in slides. This will require the use of computer vision libraries such as OpenCV. The idea is to identify unique characteristics in each slide and detect when these change significantly.

In [ ]:
import cv2
import numpy as np

def find_key_features(image):
    # Verifica il numero di canali dell'immagine
    if len(image.shape) == 3 and image.shape[2] == 3:
        # L'immagine è a colori, convertila in scala di grigi
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        # L'immagine è già in scala di grigi
        gray = image

    # Applica il rilevamento dei bordi
    edges = cv2.Canny(gray, 50, 150)
    # Trova i contorni
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours


def compare_slides(slide1, slide2, threshold=0.7):
    # Compare the key features of two slides
    slide1_features = find_key_features(slide1)
    slide2_features = find_key_features(slide2)

    # Verifica che almeno uno dei frame abbia dei contorni rilevati
    max_len_features = max(len(slide1_features), len(slide2_features))
    if max_len_features == 0:
        return False  # Entrambi i frame non hanno contorni, quindi non confrontarli

    # Example comparison logic: Compare the number of contours
    if abs(len(slide1_features) - len(slide2_features)) / max_len_features > threshold:
        return True
    return False

def save_frame(frame, timestamp, output_folder='slides'):
    print(sum_time(timestamp, start_time)) ## WHERE sum_time IS DEFINED?
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    filename = f"{output_folder}/slide_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"Saved: {filename}")

def milliseconds_to_hh_mm_ss(milliseconds):
    hours = int(milliseconds / 3600000)
    minutes = int((milliseconds % 3600000) / 60000)
    seconds = int((milliseconds % 60000) / 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"



In [ ]:
# Slides detection

# Threshold value determines how different two slides must be to be considered
# different from each other. In this specific case, the value of 0.7 is used
# to determine the relative difference in the number of contours between two slides

def detect_slides(video_path, threshold=0.05, skip_frames=30):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = []

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            if compare_slides(prev_frame, curr_frame, threshold):
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp_readable = milliseconds_to_hh_mm_ss(timestamp_ms)
                slide_changes.append(timestamp_readable)
                save_frame(curr_frame, timestamp_readable)  # Save the current slide
                prev_frame = curr_frame
                print(f"Slide change detected at: {timestamp_readable}")

        frame_count += 1

    cap.release()
    return slide_changes

slide_changes = detect_slides(input_video_path)

00:00:01
Saved: slides/slide_00:00:01.jpg
Slide change detected at: 00:00:01
00:00:02
Saved: slides/slide_00:00:02.jpg
Slide change detected at: 00:00:02
00:00:04
Saved: slides/slide_00:00:04.jpg
Slide change detected at: 00:00:04
00:00:06
Saved: slides/slide_00:00:06.jpg
Slide change detected at: 00:00:06
00:00:09
Saved: slides/slide_00:00:09.jpg
Slide change detected at: 00:00:09
00:00:10
Saved: slides/slide_00:00:10.jpg
Slide change detected at: 00:00:10
00:00:15
Saved: slides/slide_00:00:15.jpg
Slide change detected at: 00:00:15
00:00:26
Saved: slides/slide_00:00:26.jpg
Slide change detected at: 00:00:26
00:00:27
Saved: slides/slide_00:00:27.jpg
Slide change detected at: 00:00:27
00:00:28
Saved: slides/slide_00:00:28.jpg
Slide change detected at: 00:00:28
00:02:33
Saved: slides/slide_00:02:33.jpg
Slide change detected at: 00:02:33
00:02:36
Saved: slides/slide_00:02:36.jpg
Slide change detected at: 00:02:36
00:02:39
Saved: slides/slide_00:02:39.jpg
Slide change detected at: 00:02:39

###CHECK IF IS A SLIDE (TO BE TESTED)
text_threshold: This threshold represents the minimum amount of area in the image that must be covered by text for the image to be considered a slide. The idea is that presentation slides typically contain a significant amount of text.

The value 100 in this case is an arbitrary example. If the calculated value of the text area in the image is less than text_threshold, the image is not considered a slide.
This parameter can be adjusted based on the specific nature of the slides you are analyzing. If your slides have a lot of text, you may want to set a higher value; however, if they contain little text, a lower value may be more appropriate.

feature_threshold: This threshold is used to evaluate the presence of other key features (such as specific shapes, layout, colors) that could identify an image as a slide.

The value 0.5 here is also an example and indicates a percentage or relative value of these characteristics. If key features detected in the image are below this threshold, the image is not considered a slide.
This parameter can also be adapted based on the specific characteristics of your video slides.

In [ ]:
# Slides detection with no-slide-sharing error prevention

# Checks if the current image has characteristics that suggest it is a slide
def is_slide(image, text_threshold=100, feature_threshold=0.5):
    # Convert the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect text areas (you could use more advanced text detection methods)
    # For now, let's use a simple thresholding method
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    text_area = cv2.countNonZero(thresh)

    # Check if the amount of text is above a certain threshold
    if text_area < text_threshold:
        return False

    # Check for key features (like specific shapes, colors, etc.)
    # For now, this is just a placeholder for your specific feature detection logic
    features = find_key_features(image)
    if len(features) < feature_threshold:
        return False

    return True

# Modify the detect_slides function to use this new logic
def detect_slides(video_path, threshold=0.7, skip_frames=30):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = []

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            if compare_slides(prev_frame, curr_frame, threshold) and is_slide(curr_frame):
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp_readable = milliseconds_to_hh_mm_ss(timestamp_ms)
                slide_changes.append(timestamp_readable)
                save_frame(curr_frame, timestamp_readable)  # Save the current slide
                prev_frame = curr_frame
                print(f"Slide change detected at: {timestamp_readable}")

        frame_count += 1

    cap.release()
    return slide_changes


###CHECKING FOR TEACHER NOTES ON SLIDES OR BLACK-BOARD (STILL TO BE TESTED!!)

In [ ]:
def frame_difference(frame1, frame2):
    # Calcola la differenza assoluta tra due frame
    diff = cv2.absdiff(frame1, frame2)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count

def detect_changes(video_path, change_threshold=500000, skip_frames=30):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    significant_changes = []

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            gray_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            if frame_difference(prev_frame, gray_frame) > change_threshold:
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp_readable = milliseconds_to_hh_mm_ss(timestamp_ms)
                significant_changes.append(timestamp_readable)
                save_frame(curr_frame, timestamp_readable)  # Salva l'immagine corrente
                prev_frame = gray_frame
                print(f"Change detected at: {timestamp_readable}")

        frame_count += 1

    cap.release()
    return significant_changes


In [ ]:
def detect_slides_and_changes(video_path, slide_threshold=0.7, change_threshold=500000, skip_frames=30):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    events = []  # Questa lista conterrà sia i cambi di slide sia le modifiche significative

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

            # Controlla sia per i cambi di slide sia per le modifiche significative
            if compare_slides(prev_frame_gray, curr_frame_gray, slide_threshold) or frame_difference(prev_frame_gray, curr_frame_gray) > change_threshold:
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp_readable = milliseconds_to_hh_mm_ss(timestamp_ms)
                events.append({'type': 'change', 'timestamp': timestamp_readable})
                save_frame(curr_frame, timestamp_readable)
                print(f"Event detected at: {timestamp_readable}")
                prev_frame_gray = curr_frame_gray

        frame_count += 1

    cap.release()
    return events


63

In [ ]:
print(f"detected slides {len(slide_changes)}")

# Audio extraction

In [ ]:
!pip install cohere
!pip install openai
!pip install git+https://github.com/openai/whisper.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-rgpj0e4h
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-rgpj0e4h
  Resol

In [ ]:
import whisper
#if transcription suck change the model size: tiny < base < small < medium < large
#if transcribing from english use .en eg small.en
def transcribe_video(video_path, model_size='medium.en'):
    model = whisper.load_model(model_size)
    result = model.transcribe(video_path)
    return result

In [ ]:
def interleave_transcription_with_slides(transcription, slide_changes):
    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes) and slide_changes[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

##NEW OPTIMIZED FUNCTION BY LessonExtractorAI (STILL TO BE TESTED!!)
To better synchronize slides with audio, it's important to ensure that transcription timing and slide change timestamps are accurate and aligned. One way to do this is to make sure the timestamps are consistent with each other and accurately reflect when a slide appears in the video.

In [ ]:
def interleave_transcription_with_slides(transcription, slide_changes):
    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Convert segment start time to a comparable format
        segment_start_seconds = convert_timestamp_to_seconds(segment['start'])

        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes) and slide_changes[current_slide] <= segment_start_seconds:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data


#pdf or docx creation

In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
from docx import Document
from docx.shared import Pt, Inches, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.section import WD_SECTION_START
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH

def create_document(interleaved_data, video_path, slides_folder='slides'):
    # Crea il documento Word
    doc = Document()
    doc.styles['Normal'].font.name = 'Calibri'
    doc.styles['Normal'].font.size = Pt(11)

    for item in interleaved_data:
        if item['type'] == 'slide_change':
            # Aggiungi l'immagine della slide
            slide_image_path = f"{slides_folder}/slide_{item['timestamp']}.jpg"
            p = doc.add_paragraph()
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER  # Centra il paragrafo
            try:
                run = p.add_run()
                run.add_picture(slide_image_path, width=Inches(6))  # Regola la larghezza se necessario
                run.add_break()  # Aggiungi una pausa dopo l'immagine
            except Exception as e:
                p.add_run(f"Could not load slide image: {slide_image_path}")
            p.add_run().add_break()  # Aggiungi una pausa prima dell'immagine successiva
        else:
            # Aggiungi il testo trascritto
            cleaned_text = item['text'].strip().replace('\n', ' ')
            p = doc.add_paragraph(cleaned_text)
            p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY  # Giustifica il testo

    # Salva il documento
    output_file = f"{os.path.splitext(os.path.basename(video_path))[0]}.docx"
    doc.save(output_file)
    print(f"Document saved as: {output_file}")

# Altre funzioni...



def convert_timestamp_to_seconds(timestamp_str):
    parts = timestamp_str.split(':')
    if len(parts) == 3:
        hours, minutes, seconds = map(int, parts)
        return hours * 3600 + minutes * 60 + seconds
    elif len(parts) == 2:
        minutes, seconds = map(int, parts)
        return minutes * 60 + seconds
    else:
        raise ValueError(f"Invalid timestamp format: {timestamp_str}")

def interleave_transcription_with_slides(transcription, slide_changes):
    # Convert slide change timestamps to seconds
    slide_changes_seconds = [convert_timestamp_to_seconds(ts) for ts in slide_changes]

    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes_seconds) and slide_changes_seconds[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes_seconds):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

# Transcribe video
transcription = transcribe_video(input_video_path)

# Interleave transcription with slide changes
interleaved_data = interleave_transcription_with_slides(transcription, slide_changes)

# Create DOCX document
create_document(interleaved_data, output_video_path)

100%|█████████████████████████████████████| 1.42G/1.42G [00:30<00:00, 49.4MiB/s]


Document saved as: gpu-20240515_converted.docx


In [ ]:
%ls

 20240404_converted.docx       lez15.mp4
 gpu-20240430_converted.docx   lez3_converted.docx
 gpu-20240430.mp4              lez3_slides_only.docx
 gpu-20240515.mp4              lez6.mp4
 gpu-20240517.mp4              lez9.docx
 gpu-20240529.mp4             'Marcello Restelli_s Personal Room-20240312 1333-1_converted.docx'
 gpu-20240531.mp4             'Marcello Restelli_s Personal Room-20240326 1330-1_converted.docx'
 LessonsExtractor.ipynb        ML20240416_converted.docx
 lez13_converted.docx          ML20240430_converted.docx
 lez13.mp4                     ML20240502_converted.docx
 lez15_converted.docx          slides/


eliminate the slides images after having generated the docx output

In [ ]:
%rm -rf /content/drive/MyDrive/lessonsExtractor/slides

code to put only slides in a docx

In [ ]:
from docx import Document
from docx.shared import Inches
import os

def create_document_with_slides_only(slide_changes, video_path, slides_folder='slides'):
    # Extract base name and remove extension to form the DOCX file name
    base_name = os.path.basename(video_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_file = f'{file_name_without_ext}_slides_only.docx'

    doc = Document()

    for timestamp in slide_changes:
        slide_image_path = f"{slides_folder}/slide_{timestamp}.jpg"
        p = doc.add_paragraph()
        try:
            p.add_run().add_picture(slide_image_path, width=Inches(6))  # Adjust the width as needed
        except Exception as e:
            p.add_run(f"Could not load slide image: {slide_image_path}")

    doc.save(output_file)
    print(f"Document saved as: {output_file}")

# Example usage:
# slide_changes = ['00:00:10', '00:01:15', '00:02:30']  # List of timestamps for slide changes
create_document_with_slides_only(slide_changes, input_video_path)


ModuleNotFoundError: No module named 'docx'